<a href="https://colab.research.google.com/github/christaniemann/engagement_lang_hwc/blob/main/Program_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install nltk==3.4

     |████████████████████████████████| 1.4 MB 5.4 MB/s 
  Created wheel for nltk: filename=nltk-3.4-py3-none-any.whl size=1436397 sha256=800aecde13a05ce37c55fd16bff4f0bd832744ce87e0cd949dda36dc6af5d068
  Stored in directory: /root/.cache/pip/wheels/13/b8/81/2349be11dd144dc7b68ab983b58cd2fae353cdc50bbdeb09d0
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [2]:
!pip install nltk

In [3]:
import nltk
nltk.__version__

'3.4'

In [4]:
#importing libraries
import csv
import re
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.util import ngrams
from nltk.stem import PorterStemmer
from numpy import mean
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
stopwords = nltk.corpus.stopwords.words('english')

#defining functions
def clean(string1):
    '''removes punctuation and capitalization'''
    string1 = str.lower(string1)
    string1 = re.sub(r"[^\w\s]|\d", r" ", string1)
    string1 = re.sub(r"  ", r" ", string1)
    return string1

def average_len(list1):
    '''computes the average length of a list of strings'''
    return sum(list1) / len(list1)

def num_average(list2):
    '''computes the numerical average of a list of integers'''
    avg = mean(list2)
    return avg 

def ngram(list1, num):
    '''returns n-grams for a given tokenized list'''
    n_grams = ngrams(list1, num)
    return [ ' '.join(grams) for grams in n_grams]

def sentiment_analyzer(appointment_form_text):
    '''returns sentiment analysis statistics'''
    return analyzer.polarity_scores(appointment_form_text)


#retrieving data from Excel
fr1 = open("/content/Sample Appointment Form Data2.csv", "r", encoding = "utf-8")
fr = csv.reader(fr1)
next(fr)


#Preparing the data: clean, tokenize, and lemmatize
lemmatizer = WordNetLemmatizer()

all_tokens = [] #list of all words, with repeats
all_meaningful_tokens = [] #list of all words without stopwords
data = [] #list of words paired with engagement score

for row in fr:
    clean_strings = clean(row[7])
    tokens = word_tokenize(clean_strings)
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    for i in lemmatized_tokens:
        all_tokens.append(i)
    data.append([int(row[4]), lemmatized_tokens]) #[engagement score, [[token1],[token2]]

for i in all_tokens:
    if i not in stopwords:
        all_meaningful_tokens.append(i)

print("\n All words: \n", sorted(all_tokens), "\n")
print("All Words Without Stopwords: \n", sorted(all_meaningful_tokens), "\n")
print("Matched Start Scores and Words: \n", data, "\n")


#BASIC STATISTICS (min, max, avg. string length and frequencies, and sentiment statistics)

print("\n \n BASIC STATISTICS \n")
print("Total words used:", len(all_tokens))
print("Total meaningful words used:", len(all_meaningful_tokens), "\n")

lengths = [] #comparing string lengths
for i in data: 
    test = len(i[1])
    lengths.append(test)
average_length = average_len(lengths)
print("Average length of strings:", average_length)
print("Minimum length of string:", min(lengths))
print("Maximum length of string:", max(lengths))

token_freq = Counter(all_tokens) #counting word frequencies with and without stopwords
print("\n Number of tokens for each word type:", token_freq, "\n")
meaningful_token_freq = Counter(all_meaningful_tokens)
print("Number of tokens for each word type (no stopwords):", meaningful_token_freq, "\n")


#Grouping data by engagement score (1-5)
not_engaged = []
less_engaged = []
neutral = []
engaged = []
very_engaged = []

for i in data:
    if(i[0] == 1):
        not_engaged.append(i)
    elif (i[0] == 2):
        less_engaged.append(i)
    elif (i[0] == 3):
        neutral.append(i)
    elif (i[0] == 4):
        engaged.append(i)
    elif (i[0] == 5):
        very_engaged.append(i)

print("Not Engaged:", not_engaged)
print("Less Engaged:", less_engaged)
print("Neutral:", neutral)
print("Engaged:", engaged)
print("Very Engaged:", very_engaged, "\n")

not_engaged_lengths = []
less_engaged_lengths = []
neutral_lengths = []
engaged_lengths = []
very_engaged_lengths = []

engagement_categories = [not_engaged, less_engaged, neutral, engaged, very_engaged]
lengths_lists = [not_engaged_lengths, less_engaged_lengths, neutral_lengths, engaged_lengths, very_engaged_lengths]
category_names = ["Not Engaged:", "Less Engaged:", "Neutral:", "Engaged:", "Very Engaged:"]

for list in engagement_categories: #calculating average string lengths in each category
    for i in list:
        test = len(i[1])
        index = engagement_categories.index(list)
        lengths_lists[index].append(test)
    average_length = average_len(lengths_lists[index])
    print("Average Length of String for", category_names[index], average_length)

#Sentiment Statistics
#Analyzing the average intensity of the sentiment (positive, negative, neutral) for appointment form strings in each category
print("\n \n Sentiment Statistics \n")
analyzer = SentimentIntensityAnalyzer()

ne_sentiments = [] #Creating lists for manipulating the sentiment scores
le_sentiments = []
n_sentiments = []
e_sentiments = []
ve_sentiments = []
sentiment_lists = [ne_sentiments, le_sentiments, n_sentiments, e_sentiments, ve_sentiments]
sentiment_list_names = ["Not engaged sentiments:", "Less engaged sentiments:", "Neutral sentiments:", "Engaged sentiments:", "Very engaged sentiments:"]

fr1.seek(0) #resetting back to beginning of file
fr1.readline() #skip line 1 with headings

for row in fr: #Adding sentiments to the appropriate category list
    sentiments = sentiment_analyzer(row[7])
    if int(row[4]) == 1:
        ne_sentiments.append(sentiments) 
    elif int(row[4]) == 2:
        le_sentiments.append(sentiments)
    elif int(row[4]) == 3:
        n_sentiments.append(sentiments)
    elif int(row[4]) == 4:
        e_sentiments.append(sentiments)
    elif int(row[4]) == 5:
        ve_sentiments.append(sentiments)

for list in sentiment_lists: #averaging the positive, negative, and neutral sentiment scores in each engagement category

    index1 = sentiment_lists.index(list)
    print(sentiment_list_names[index1], "\n")

    negs = []
    neus = []
    poss = []
    compounds = []

    for d in list:
        negs.append(d['neg'])
        neus.append(d['neu'])
        poss.append(d['pos'])
        compounds.append(d['compound'])
    print("Negative:", num_average(negs))
    print("Neutral:", num_average(neus))
    print("Positive:", num_average(poss))
    print("Compound:", num_average(compounds), "\n \n")


#SINGLE WORD CATEGORY ANALYSIS
print("\n \n \n SINGLE WORD CATEGORY ANALYSIS \n")

checkbox_categories = ["evidence", "source", "idea", "organization", "structure", "convention", "transition", "flow", "argument", "vocabulary", "tone", "style", "grammar", "mechanic", "citation", "format"]
verbs = ["write", "edit", "help", "check", "correct", "collaborate", "discuss", "refine", "revise", "correct", "fix", "error", "mistake", "proofread", "rewrite", "draft", "outline", "plan", "brainstorm"]

for list in engagement_categories:
    
    index = engagement_categories.index(list)
    print(category_names[index])

    temp_list = [] #flattening multidimentional lists
    for i in list: 
        temp_list.append(i[1])
    flat = [item for sublist in temp_list for item in sublist]

    flat2 = [] #deleting stopwords
    for word in flat: 
        if word not in stopwords:
            flat2.append(word)
    print(sorted(flat2), "\n")

    freq = Counter(flat2) #counting word frequencies in each category
    print(freq, "\n")

    print("Checkbox Categories:") #tallying checkbox focuses in each category
    expressed_checkboxes = []
    for word in checkbox_categories:
        if word in flat2:
            expressed_checkboxes.append(word)
    print(Counter(expressed_checkboxes), "\n")

    print("Verbs:") #tallying verbs in each category
    expressed_verbs = []
    for word in verbs:
        if word in flat2:
            expressed_verbs.append(word)
    print(Counter(expressed_verbs), "\n")

    print("Pronouns:") #tallying pronouns in each category
    expressed_pronouns = []
    for i in flat:
        if i in ["you", "i", "we", "us"]:
            expressed_pronouns.append(i)
    print(Counter(expressed_pronouns), "\n \n")


#PHRASAL N-GRAM CATEGORY ANALYSIS
print("\n PHRASAL (N-GRAM) CATEGORY ANALYSIS \n")
phrases = ["word choice", "just want someone", "just want", "just wan", "look over", "make sense", "make sure", "i need help", "can you", "can we", "could you", "could we", "i want", "i 'm not sure if", "please fix", "please check", "please correct"]

#Bi-gram and tri-gram analysis

for list in engagement_categories:

    index = engagement_categories.index(list)
    print(category_names[index], "\n")

    #organic phrases
    temp_bigrams = [] #bigram analysis
    for i in list:
        bigrams = ngram(i[1], 2)
        for gram in bigrams:
            temp_bigrams.append(gram)
    bigram_freq = Counter(temp_bigrams)
    print("Organic Phrases:")
    print("bigrams:", bigram_freq)

    temp_trigrams = [] #trigram analysis
    for i in list:
        trigrams = ngram(i[1], 3)
        for gram in trigrams:
            temp_trigrams.append(gram)
    trigram_freq = Counter(temp_trigrams)
    print("trigrams: ", trigram_freq, "\n")

    #Pre-identified phrases
    expressed_phrases = []
    for i in temp_bigrams:
        if i in phrases:
            expressed_phrases.append(i)
    print("Pre-identified phrases: ")
    print(Counter(expressed_phrases), "\n \n")

#VERB ANALYSIS
#MANOVA test will determine whether differences in verb-type usage are statisticalaly significant

#stemming data and verb lists for easy identification
ps = PorterStemmer()

passive_engagement_verbs = ["edit", "check", "fix", "correct", "proofread"]
active_engagement_verbs = ["discuss", "revise", "rework", "restructure", "review", "collaborate", "analyze" "refine", "rewrite", "draft", "outline", "plan", "brainstorm", "develop", "improve", "learn", "structure"]

stemmed_Pverbs = [ps.stem(word) for word in passive_engagement_verbs]
stemmed_Averbs = [ps.stem(word) for word in active_engagement_verbs]
print(stemmed_Pverbs)
print(stemmed_Averbs, "\n")

all_stemmed = []
data_stemmed = []

fr1.seek(0)
fr1.readline()
for row in fr:
    new_line = re.sub("wanna", "", row[7]) #removing "wanna" from obscuring data
    clean_strings = clean(new_line)
    tokens = word_tokenize(clean_strings)
    stemmed_tokens = [ps.stem(word) for word in tokens]
    for i in stemmed_tokens:
        all_stemmed.append(i)
    data_stemmed.append([int(row[4]), stemmed_tokens])

ne_stemmed = []
le_stemmed = []
n_stemmed = []
e_stemmed = []
ve_stemmed = []
stemmed_lists = [ne_stemmed, le_stemmed, n_stemmed, e_stemmed, ve_stemmed]

for i in data_stemmed: #adding stemmed data to the appropriate lists
    if(i[0] == 1):
        ne_stemmed.append(i)
    elif (i[0] == 2):
        le_stemmed.append(i)
    elif (i[0] == 3):
        n_stemmed.append(i)
    elif (i[0] == 4):
        e_stemmed.append(i)
    elif (i[0] == 5):
        ve_stemmed.append(i)

# Get part of speech tags to ensure above verb forms are actually syntactic verbs
for list in stemmed_lists:

    index = stemmed_lists.index(list)
    print(category_names[index], "\n")

    expressed_Pverbs = []
    kept_Pverbs = []
    expressed_Averbs = []
    kept_Averbs = []

    for i in list:
        tags = nltk.pos_tag(i[1]) #list 'tags' gets rewritten for each engagement category

        for i in tags: #if term contains a passive engagement verb, add it to expressed list
            if i[0] in stemmed_Pverbs:
                expressed_Pverbs.append(i)
                if i[1] in ["VB", "VBP", "VGB"]: #if term is not a verb (VB, VBP, VBG), don't add to kept list
                    kept_Pverbs.append(i[0])

        for i in tags: #if term contains an active engagement verb, add it to expressed list
            if i[0] in stemmed_Averbs: 
                expressed_Averbs.append(i)
                if i[1] in ["VB", "VBP", "VBG"]: #if term is not a verb (VB, VBP, VBG), don't add to kept list
                    kept_Averbs.append(i[0])
    
    print("Passive Engagement Verbs:", kept_Pverbs)
    print("Active Engagement Verbs:", kept_Averbs, "\n")



fr1.close()

     |████████████████████████████████| 125 kB 5.3 MB/s 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.



 All words: 
 ['a', 'a', 'a', 'a', 'a', 'about', 'active', 'addressing', 'all', 'also', 'am', 'am', 'am', 'am', 'an', 'analysis', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'any', 'apa', 'are', 'assignment', 'assignment', 'assignment', 'assignment', 'at', 'at', 'bad', 'being', 'between', 'bio', 'body', 'brainstorm', 'business', 'business', 'but', 'changing', 'check', 'choice', 'choice', 'citation', 'citation', 'cited', 'clear', 'clear', 'comment', 'concise', 'conclusion', 'conclusion', 'consultant', 'could', 'cover', 'd', 'didn', 'don', 'edit', 'edits', 'english', 'english', 'enough', 'error', 'essay', 'everything', 'everything', 'everything', 'feedback', 'figure', 'fix', 'fixing', 'flow', 'flow', 'flow', 'flow', 'for', 'for', 'for', 'formatting', 'gave', 'general', 'get', 'give', 'given', 'good', 'grade', 'graduate', 'grammar', 'grammar', 'grammar', 'grammar', 'grammar', 'ha', 'have', 'he', 'help', 'help', 'help', 'help', 